In [1]:
import requests
import re
from bs4 import BeautifulSoup
import pandas as pd
import os


ModuleNotFoundError: No module named 'pandas'

In [ ]:
url1='https://www.lyrics.com/artist/Taylor-Swift/816977'
url2='https://www.lyrics.com/artist/Ed-Sheeran/2342870'

# exploration

In [ ]:
response1=requests.get(url1)
response2=requests.get(url2)
print(response1.status_code, response2.status_code)

In [ ]:
responses=[response1,response2]

In [ ]:
file1 = "TaylorSwift/html/TS_lyrics.html"
file2 = 'EdSheeran/html/ED_lyrics.html'
files=[file1,file2]

In [ ]:
for file,response in zip(files,responses):
    with open(file, "w") as afile:
        afile.write(response.text)

In [ ]:
ls TaylorSwift/html/

In [ ]:
html_text=[]
for file,response in zip(files,responses):
    with open(file, "r") as afile:
        html_text.append(afile.read())

In [ ]:
html_text[1]

In [ ]:
re.findall("href\=\"/lyric/\w+",html_text[1])#\=\"lyric/",html_text[1])

In [ ]:
soup = BeautifulSoup(response1.text)
type(soup), type(response1.text)

In [ ]:
print(soup.prettify())

In [ ]:
hyperlinks = soup.find_all('a')
hyperlinks

# GET Lyrics : BS - takes a long time, run only once

In [ ]:
def get_lyrics(artist_name,songs):
    for hyperlink, song_number in zip(songs.href,songs.song_number):
        main_path=f'{artist_name}/lyrics'
        lyrics_soup =BeautifulSoup(requests.get(hyperlink).text)
        # Look for the tag `pre` and id 'lyric-body-text'
        try:
            #print(f'writing file {song_number}')
            lyrics_text = lyrics_soup.find("pre", id="lyric-body-text").text  # catch the id / use for class
        except:
            print('didnt find song lyrics in html file')
            continue

        with open(f'{main_path}/{song_number}.txt', "w") as afile:
            afile.write(lyrics_text)

In [ ]:
def songs_lyrics_url(artist_name,artist_code):
    #for artist_name,artist_code in zip(artist,code):

    #CREATE ARTIST FOLDER IF DOES NOT EXIST
    if os.path.exists(artist_name)==False: 
        os.mkdir(artist_name)
        os.mkdir(artist_name+'/html')
        os.mkdir(artist_name+'/lyrics')
    main_url = "https://www.lyrics.com"
    artist_url = "artist"
    full_url = f'{main_url}/{artist_url}/{artist_name}/{artist_code}'

    #CREATE HTML FILE IF DOES NOT EXIST
    html_file=artist_name+f'/html/{artist_name}.html'

    if os.path.exists(html_file)==False: 
        response=requests.get(full_url)
        with open(html_file,'w') as file: 
            file.write(response.text)
    with open(html_file, "r") as file:
        html = file.read()

    #NOW, USE BS TO DO THE WORK
    soup = BeautifulSoup(html)
    # Get all hrefs
    empty_list = []
    hrefs = soup.find_all('a')

    for href in hrefs:  #hyperlink
        my_href = href["href"]
        if "lyric/" in my_href:
            empty_list.append(f'{main_url}{my_href}')

    #FILTERING SONGS THAT REPEAT BASED ON UNIQUE SONG NAME 
    songs=pd.DataFrame()
    for x in empty_list:
        song_string=re.findall('\d{8}',x)
        song_name=re.findall('[^.\/]+$',x)
        if len(song_string)>0 and len(song_name)>0:
            songs=songs.append({
                'artist':artist_name,
                'song_number':int(eval(song_string[0])),
                'song_name':str(song_name[0]),
                'href':x
                },ignore_index=True)
    print(songs.head(3))
    unique_songs=songs['song_name'].str.split(' ',expand=True).stack().nunique()
    print(f'Unique songs by {artist_name}: {unique_songs}')
    songs=songs.groupby('song_name').first().reset_index()
    #NOW, WRITE ALL LYRICS TO FILES
    get_lyrics(artist_name,songs)

In [ ]:
artists=['Taylor+Swift','Ed+Sheeran']#,'Miley+Cyrus']
codes=[816977,2342870]#,823418]
all_href=pd.DataFrame()
for artist,code in zip(artists,codes):
    #all_href=all_href.append(
    print(artist,code)
    songs_lyrics_url(artist,code)


Taylor+Swift 816977
         artist  song_number                 song_name  \
0  Taylor+Swift   35948709.0  Look+What+You+Made+Me+Do   
1  Taylor+Swift   35965938.0                Love+Story   
2  Taylor+Swift   35965923.0                      Mine   

                                                href  
0  https://www.lyrics.com/lyric/35948709/Taylor+S...  
1  https://www.lyrics.com/lyric/35965938/Taylor+S...  
2  https://www.lyrics.com/lyric/35965923/Taylor+S...  
Unique songs by Taylor+Swift: 362
didnt find song lyrics in html file
didnt find song lyrics in html file
didnt find song lyrics in html file
didnt find song lyrics in html file
didnt find song lyrics in html file
didnt find song lyrics in html file
didnt find song lyrics in html file
didnt find song lyrics in html file
Ed+Sheeran 2342870
       artist  song_number         song_name  \
0  Ed+Sheeran   36137186.0  Lay+It+All+on+Me   
1  Ed+Sheeran   36131004.0  Lay+It+All+on+Me   
2  Ed+Sheeran   36370642.0              BL

# Creating the corpus

In [ ]:
import glob

In [ ]:
def substitute(lyrics):
    return(re.sub(r'\W',' ',lyrics))

In [ ]:
import tqdm

artists=['Taylor+Swift','Ed+Sheeran']#,'Miley+Cyrus']
corpus = pd.DataFrame()

for artist_name in tqdm.tqdm(artists):
    lyrics_files = glob.glob(f"{artist_name}/lyrics/*.txt")
    for file in lyrics_files:
        with open(file,'r') as afile:
            new_lyrics=afile.read()
            corpus=corpus.append({
                'lyrics':new_lyrics,
                'artist':artist_name
                },ignore_index=True)
            corpus=corpus[~corpus['lyrics'].str.startswith('<')]
            corpus['lyrics']=corpus['lyrics'].apply(substitute)

corpus

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:38<00:00, 19.40s/it]


,lyrics,artist
0,There s somethin bout the way The street look...,Taylor+Swift
1,How many days did I spend Thinkin bout how d...,Taylor+Swift
2,I m like the water when your ship rolled in th...,Taylor+Swift
3,Combat I m ready for combat I say I don t wan...,Taylor+Swift
4,Once upon a time I believe it was a Tuesday w...,Taylor+Swift
...,...,...
579,I met this girl late last year She said don t ...,Ed+Sheeran
580,I m not a rapper I m a singer with a flow I v...,Ed+Sheeran
581,They say boys don t cry But your dad has shed ...,Ed+Sheeran
582,It s been a while since we ve been alone So tu...,Ed+Sheeran


# Modelling

In [ ]:
corpus=corpus.reset_index()

In [ ]:
X=corpus['lyrics']
y=corpus['artist']

In [ ]:
X.shape

(584,)

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y,random_state=11,stratify=y)

In [ ]:
type(X_train.to_numpy())

numpy.ndarray

In [ ]:
X_train=X_train.to_numpy()
y_train=y_train.to_numpy()
X_test=X_test.to_numpy()
y_test=y_test.to_numpy()


In [ ]:
print(X_train.shape, y_train.shape, X_test.shape, y_test.shape)

(438,) (438,) (146,) (146,)


In [ ]:
X_train

array(['Say you re sorry That face of an angel Comes out just when you need it to As I paced back and forth all this time  Cause I honestly believed in you Holding on The days drag on Stupid girl I should have known  I should have known  That I m not a princess  this ain t a fairy tale I m not the one you ll sweep off her feet Lead her up the stairwell This ain t Hollywood  this is a small town I was a dreamer before you went and let me down Now it s too late for you And your white horse  to come around  Maybe I was naive  Got lost in your eyes And never really had a chance My mistake I didn t know to be in love You had to fight to have the upper hand I had so many dreams About you and me Happy endings Now I know  I m not a princess  this ain t a fairy tale I m not the one you ll sweep off her feet Lead her up the stairwell This ain t Hollywood  this is a small town I was a dreamer before you went and let me down Now it s too late for you And your white horse  to come around  And there

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

In [ ]:
vectorizer = CountVectorizer()#stop_words='english') # instanciate the CV
X_new= vectorizer.fit_transform(X_train) # transform
X_df = pd.DataFrame(X_new.todense(), columns=vectorizer.get_feature_names_out(), index=y_train)
X_df



,000,0121,0161,03,10,12,12s,16th,17,18,...,zion,zone,zoo,zoom,ça,éloges,époque,équipe,étais,était
Taylor+Swift,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Taylor+Swift,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Taylor+Swift,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Ed+Sheeran,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Taylor+Swift,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Ed+Sheeran,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Taylor+Swift,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Taylor+Swift,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Ed+Sheeran,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
vectorizer.get_feature_names_out()

array(['000', '0121', '0161', ..., 'équipe', 'étais', 'était'],
      dtype=object)

# TF-IDF

In [ ]:
from sklearn.feature_extraction.text import TfidfTransformer

In [ ]:
tf = TfidfTransformer() 
#Use the CountVectorised data
X_norm = tf.fit_transform(X_df)
X_norm

<438x5987 sparse matrix of type '<class 'numpy.float64'>'
	with 52995 stored elements in Compressed Sparse Row format>

In [ ]:
X_norm_df=pd.DataFrame(X_norm.todense(), columns=vectorizer.get_feature_names_out(), index=y_train)
X_norm_df

,000,0121,0161,03,10,12,12s,16th,17,18,...,zion,zone,zoo,zoom,ça,éloges,époque,équipe,étais,était
Taylor+Swift,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Taylor+Swift,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Taylor+Swift,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Ed+Sheeran,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Taylor+Swift,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Ed+Sheeran,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Taylor+Swift,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Taylor+Swift,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Ed+Sheeran,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


# LR

In [ ]:
from sklearn.linear_model import LogisticRegression

In [ ]:
m = LogisticRegression()

m.fit(X_new, y_train)
print(f'score for LR train data: {m.score(X_new,y_train)}')
print(f'score for LR test data: {m.score(X_test_new,y_test)}')

score for LR train data: 0.9908675799086758
score for LR train data: 0.8561643835616438


/Users/prapti/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [ ]:
m.predict_proba(X_new)

array([[1.29311783e-02, 9.87068822e-01],
       [2.89702884e-03, 9.97102971e-01],
       [2.04407766e-04, 9.99795592e-01],
       [9.98254757e-01, 1.74524333e-03],
       [1.79641384e-03, 9.98203586e-01],
       [1.42897375e-02, 9.85710263e-01],
       [1.93547057e-03, 9.98064529e-01],
       [9.98718522e-01, 1.28147763e-03],
       [6.87091731e-04, 9.99312908e-01],
       [7.97487539e-05, 9.99920251e-01],
       [4.51900006e-03, 9.95481000e-01],
       [1.33920957e-02, 9.86607904e-01],
       [9.99008509e-01, 9.91491115e-04],
       [1.21160783e-05, 9.99987884e-01],
       [9.76794447e-01, 2.32055533e-02],
       [9.99910158e-01, 8.98417098e-05],
       [9.84880384e-01, 1.51196157e-02],
       [1.10319722e-02, 9.88968028e-01],
       [1.16753241e-02, 9.88324676e-01],
       [7.91556610e-05, 9.99920844e-01],
       [8.17508601e-03, 9.91824914e-01],
       [8.66614540e-03, 9.91333855e-01],
       [6.09097269e-06, 9.99993909e-01],
       [4.76441102e-04, 9.99523559e-01],
       [1.578136

In [ ]:
X_test_new= vectorizer.transform(X_test)

In [ ]:
m.predict(X_test_new)

array(['Taylor+Swift', 'Ed+Sheeran', 'Taylor+Swift', 'Taylor+Swift',
       'Taylor+Swift', 'Taylor+Swift', 'Taylor+Swift', 'Taylor+Swift',
       'Ed+Sheeran', 'Ed+Sheeran', 'Taylor+Swift', 'Taylor+Swift',
       'Taylor+Swift', 'Taylor+Swift', 'Ed+Sheeran', 'Ed+Sheeran',
       'Ed+Sheeran', 'Taylor+Swift', 'Ed+Sheeran', 'Taylor+Swift',
       'Ed+Sheeran', 'Taylor+Swift', 'Taylor+Swift', 'Taylor+Swift',
       'Taylor+Swift', 'Ed+Sheeran', 'Taylor+Swift', 'Taylor+Swift',
       'Taylor+Swift', 'Taylor+Swift', 'Taylor+Swift', 'Ed+Sheeran',
       'Ed+Sheeran', 'Taylor+Swift', 'Taylor+Swift', 'Ed+Sheeran',
       'Taylor+Swift', 'Taylor+Swift', 'Taylor+Swift', 'Ed+Sheeran',
       'Ed+Sheeran', 'Taylor+Swift', 'Ed+Sheeran', 'Taylor+Swift',
       'Ed+Sheeran', 'Ed+Sheeran', 'Ed+Sheeran', 'Ed+Sheeran',
       'Taylor+Swift', 'Taylor+Swift', 'Taylor+Swift', 'Ed+Sheeran',
       'Taylor+Swift', 'Taylor+Swift', 'Taylor+Swift', 'Taylor+Swift',
       'Ed+Sheeran', 'Taylor+Swift', 'Ed+Sheer

In [ ]:
y_test

array(['Taylor+Swift', 'Taylor+Swift', 'Taylor+Swift', 'Taylor+Swift',
       'Taylor+Swift', 'Taylor+Swift', 'Taylor+Swift', 'Taylor+Swift',
       'Ed+Sheeran', 'Ed+Sheeran', 'Taylor+Swift', 'Taylor+Swift',
       'Taylor+Swift', 'Taylor+Swift', 'Ed+Sheeran', 'Taylor+Swift',
       'Ed+Sheeran', 'Ed+Sheeran', 'Ed+Sheeran', 'Ed+Sheeran',
       'Ed+Sheeran', 'Ed+Sheeran', 'Taylor+Swift', 'Taylor+Swift',
       'Taylor+Swift', 'Taylor+Swift', 'Taylor+Swift', 'Taylor+Swift',
       'Taylor+Swift', 'Taylor+Swift', 'Ed+Sheeran', 'Ed+Sheeran',
       'Ed+Sheeran', 'Taylor+Swift', 'Taylor+Swift', 'Ed+Sheeran',
       'Taylor+Swift', 'Taylor+Swift', 'Ed+Sheeran', 'Taylor+Swift',
       'Ed+Sheeran', 'Taylor+Swift', 'Ed+Sheeran', 'Taylor+Swift',
       'Ed+Sheeran', 'Ed+Sheeran', 'Ed+Sheeran', 'Ed+Sheeran',
       'Taylor+Swift', 'Taylor+Swift', 'Taylor+Swift', 'Ed+Sheeran',
       'Taylor+Swift', 'Ed+Sheeran', 'Taylor+Swift', 'Taylor+Swift',
       'Ed+Sheeran', 'Taylor+Swift', 'Ed+Sheeran',

## Predict a new song

In [ ]:
song = ["She floats down the aisle like a pageant queen",'My bad habits lead to you']

In [ ]:
counts = vectorizer.transform(song)

In [ ]:
m.predict_proba(counts)

array([[0.46357938, 0.53642062],
       [0.60195107, 0.39804893]])

In [ ]:
m.predict(counts)

array(['Taylor+Swift', 'Ed+Sheeran'], dtype=object)

# Random Forest

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix


In [ ]:
rf_clf = RandomForestClassifier(max_depth=3, n_estimators=20, random_state=11)

In [ ]:
X_train, X_val, y_train, y_val = train_test_split(X, y, random_state=10, stratify=y)

In [ ]:
y.value_counts(normalize=True), y_train.value_counts(normalize=True), y_val.value_counts(normalize=True)

(Taylor+Swift    0.606164
 Ed+Sheeran      0.393836
 Name: artist, dtype: float64,
 Taylor+Swift    0.605023
 Ed+Sheeran      0.394977
 Name: artist, dtype: float64,
 Taylor+Swift    0.609589
 Ed+Sheeran      0.390411
 Name: artist, dtype: float64)

In [ ]:
X_train=X_train.to_numpy()
y_train=y_train.to_numpy()
X_val=X_val.to_numpy()
y_val=y_val.to_numpy()

In [ ]:
vectorizer = CountVectorizer()#stop_words='english') # instanciate the CV
X_new= vectorizer.fit_transform(X_train) # transform

In [ ]:
rf_clf.fit(X_new, y_train)

RandomForestClassifier(max_depth=3, n_estimators=20, random_state=11)

In [ ]:
X_val_new=vectorizer.transform(X_val)

In [ ]:
rf_predictions = rf_clf.predict(X_val_new)

In [ ]:
print(f'score for RF train data: {rf_clf.score(X_new,y_train)}')
print(f'score for RF train data: {rf_clf.score(X_val_new,y_val)}')

score for RF train data: 0.7374429223744292
score for RF train data: 0.7054794520547946


In [ ]:
rf_clf.predict(X_val_new)

array(['Taylor+Swift', 'Taylor+Swift', 'Taylor+Swift', 'Taylor+Swift',
       'Ed+Sheeran', 'Ed+Sheeran', 'Taylor+Swift', 'Ed+Sheeran',
       'Taylor+Swift', 'Taylor+Swift', 'Taylor+Swift', 'Taylor+Swift',
       'Taylor+Swift', 'Taylor+Swift', 'Taylor+Swift', 'Taylor+Swift',
       'Taylor+Swift', 'Taylor+Swift', 'Taylor+Swift', 'Taylor+Swift',
       'Taylor+Swift', 'Taylor+Swift', 'Taylor+Swift', 'Taylor+Swift',
       'Taylor+Swift', 'Taylor+Swift', 'Taylor+Swift', 'Taylor+Swift',
       'Ed+Sheeran', 'Taylor+Swift', 'Taylor+Swift', 'Taylor+Swift',
       'Taylor+Swift', 'Taylor+Swift', 'Taylor+Swift', 'Taylor+Swift',
       'Taylor+Swift', 'Taylor+Swift', 'Taylor+Swift', 'Ed+Sheeran',
       'Taylor+Swift', 'Taylor+Swift', 'Taylor+Swift', 'Taylor+Swift',
       'Taylor+Swift', 'Taylor+Swift', 'Taylor+Swift', 'Ed+Sheeran',
       'Taylor+Swift', 'Taylor+Swift', 'Taylor+Swift', 'Taylor+Swift',
       'Taylor+Swift', 'Taylor+Swift', 'Taylor+Swift', 'Taylor+Swift',
       'Taylor+Swi

In [ ]:
y_val

array(['Taylor+Swift', 'Ed+Sheeran', 'Taylor+Swift', 'Taylor+Swift',
       'Ed+Sheeran', 'Ed+Sheeran', 'Taylor+Swift', 'Ed+Sheeran',
       'Taylor+Swift', 'Taylor+Swift', 'Taylor+Swift', 'Taylor+Swift',
       'Taylor+Swift', 'Taylor+Swift', 'Ed+Sheeran', 'Ed+Sheeran',
       'Taylor+Swift', 'Ed+Sheeran', 'Taylor+Swift', 'Taylor+Swift',
       'Taylor+Swift', 'Taylor+Swift', 'Ed+Sheeran', 'Taylor+Swift',
       'Taylor+Swift', 'Taylor+Swift', 'Taylor+Swift', 'Ed+Sheeran',
       'Ed+Sheeran', 'Taylor+Swift', 'Ed+Sheeran', 'Taylor+Swift',
       'Taylor+Swift', 'Ed+Sheeran', 'Taylor+Swift', 'Taylor+Swift',
       'Ed+Sheeran', 'Taylor+Swift', 'Taylor+Swift', 'Ed+Sheeran',
       'Taylor+Swift', 'Taylor+Swift', 'Taylor+Swift', 'Ed+Sheeran',
       'Taylor+Swift', 'Ed+Sheeran', 'Taylor+Swift', 'Ed+Sheeran',
       'Taylor+Swift', 'Ed+Sheeran', 'Ed+Sheeran', 'Taylor+Swift',
       'Ed+Sheeran', 'Taylor+Swift', 'Ed+Sheeran', 'Ed+Sheeran',
       'Taylor+Swift', 'Taylor+Swift', 'Ed+Sheeran

In [ ]:
## Predict a new song

song = ["She floats down the aisle like a pageant queen",'My bad habits lead to you']

counts = vectorizer.transform(song)

rf_clf.predict_proba(counts)

rf_clf.predict(counts)

array(['Taylor+Swift', 'Taylor+Swift'], dtype=object)

# Tokenisation and lemmatization

In [ ]:
import nltk 
import string

from nltk.stem import WordNetLemmatizer
from nltk.tokenize import TreebankWordTokenizer
from nltk.tokenize import word_tokenize

from sklearn.feature_extraction import _stop_words

lemmatizer = WordNetLemmatizer()
tokenizer= TreebankWordTokenizer()

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer, tokenize_lemmatize
vectorizer = TfidfVectorizer(tokenizer=tokenize_lemmatize, stop_words=None)

ImportError: cannot import name 'tokenize_lemmatize' from 'sklearn.feature_extraction.text' (/Users/prapti/opt/anaconda3/lib/python3.9/site-packages/sklearn/feature_extraction/text.py)

In [ ]:
X= vectorizer.fit_transform(corpus) # transform
X_df = pd.DataFrame(X.todense(), columns=vectorizer.get_feature_names_out(), index=labels)
X_df

In [ ]:
vectorizer.get_feature_names_out()

# Naive Baise

In [ ]:
from sklearn.naive_bayes import MultinomialNB

m = MultinomialNB()
m.fit(X_new, y_train)
m.score(X_new, y_train)

0.9748858447488584

In [ ]:
print(f'score for NB train data: {m.score(X_new,y_train)}')
print(f'score for NB train data: {m.score(X_val_new,y_val)}')

score for NB train data: 0.9748858447488584
score for NB train data: 0.8424657534246576


In [ ]:
m.predict(X_new)

array(['Taylor+Swift', 'Taylor+Swift', 'Taylor+Swift', 'Taylor+Swift',
       'Ed+Sheeran', 'Ed+Sheeran', 'Taylor+Swift', 'Taylor+Swift',
       'Ed+Sheeran', 'Taylor+Swift', 'Ed+Sheeran', 'Ed+Sheeran',
       'Ed+Sheeran', 'Taylor+Swift', 'Ed+Sheeran', 'Taylor+Swift',
       'Ed+Sheeran', 'Taylor+Swift', 'Taylor+Swift', 'Ed+Sheeran',
       'Taylor+Swift', 'Ed+Sheeran', 'Taylor+Swift', 'Ed+Sheeran',
       'Taylor+Swift', 'Taylor+Swift', 'Ed+Sheeran', 'Taylor+Swift',
       'Taylor+Swift', 'Taylor+Swift', 'Ed+Sheeran', 'Taylor+Swift',
       'Taylor+Swift', 'Taylor+Swift', 'Taylor+Swift', 'Ed+Sheeran',
       'Ed+Sheeran', 'Ed+Sheeran', 'Taylor+Swift', 'Taylor+Swift',
       'Ed+Sheeran', 'Ed+Sheeran', 'Taylor+Swift', 'Taylor+Swift',
       'Ed+Sheeran', 'Ed+Sheeran', 'Taylor+Swift', 'Taylor+Swift',
       'Taylor+Swift', 'Ed+Sheeran', 'Ed+Sheeran', 'Ed+Sheeran',
       'Taylor+Swift', 'Ed+Sheeran', 'Taylor+Swift', 'Ed+Sheeran',
       'Taylor+Swift', 'Taylor+Swift', 'Taylor+Swift', '

In [ ]:
y_train

array(['Taylor+Swift', 'Taylor+Swift', 'Taylor+Swift', 'Taylor+Swift',
       'Ed+Sheeran', 'Ed+Sheeran', 'Taylor+Swift', 'Taylor+Swift',
       'Ed+Sheeran', 'Taylor+Swift', 'Ed+Sheeran', 'Ed+Sheeran',
       'Ed+Sheeran', 'Taylor+Swift', 'Ed+Sheeran', 'Ed+Sheeran',
       'Ed+Sheeran', 'Taylor+Swift', 'Taylor+Swift', 'Ed+Sheeran',
       'Taylor+Swift', 'Ed+Sheeran', 'Taylor+Swift', 'Ed+Sheeran',
       'Taylor+Swift', 'Taylor+Swift', 'Ed+Sheeran', 'Taylor+Swift',
       'Taylor+Swift', 'Taylor+Swift', 'Ed+Sheeran', 'Taylor+Swift',
       'Taylor+Swift', 'Taylor+Swift', 'Taylor+Swift', 'Ed+Sheeran',
       'Ed+Sheeran', 'Ed+Sheeran', 'Taylor+Swift', 'Taylor+Swift',
       'Ed+Sheeran', 'Ed+Sheeran', 'Taylor+Swift', 'Taylor+Swift',
       'Ed+Sheeran', 'Ed+Sheeran', 'Taylor+Swift', 'Taylor+Swift',
       'Taylor+Swift', 'Ed+Sheeran', 'Ed+Sheeran', 'Ed+Sheeran',
       'Taylor+Swift', 'Ed+Sheeran', 'Taylor+Swift', 'Ed+Sheeran',
       'Taylor+Swift', 'Taylor+Swift', 'Taylor+Swift', 'Ed

In [ ]:
tf.transform(cv.transform(X_test)).todense()

NameError: name 'cv' is not defined

In [ ]:
test_vec = cv.transform(X_test) # using count vectorizer
test_vec2 = tf.transform(X_test) # Using tf-idf
test_vec3 = tf.transform(test_vec) # using both
m.predict(test_vec), m.predict(test_vec2), m.predict(test_vec3)

# Stemming

# Extras

In [ ]:
#print(lyrics_files)
#         with open('all_lyrics.txt', 'w') as outfile:
#             for names in lyrics_files:
#                 with open(names) as infile:
#                     outfile.write(infile.read())
#                     outfile.write("\n")
    #all_lyrics=all_lyrics.append({
     #   'artist':artist_name,
      #  'lyrics':lyrics
    #})

In [ ]:
with open('all_lyrics.txt','r') as file:
    x=file.read()
print(x[0:100])